In [ ]:
#|default_exp icdar_data

In [ ]:
#| export
from dataclasses import dataclass

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def remove_label_and_nl(line: str):
    return line.strip()[14:]

In [ ]:
#| export
def normalized_ed(ed: int, 
                  ocr: str, 
                  gs: str):
    """Returns the normalized editdistance"""
    score = 0.0
    l = max(len(ocr), len(gs))
    if l > 0:
        score = ed / l
    return score

In [ ]:
#| export
@dataclass
class AlignedToken:
    ocr: str
    gs: str
    ocr_aligned: str
    gs_aligned: str
    start: int
    len_ocr: int

In [ ]:
#| export
@dataclass
class InputToken:
    ocr: str
    gs: str
    start: int
    len_ocr: int
    label: int

In [ ]:
#| export
def get_input_tokens(aligned_token):
    if aligned_token.ocr == aligned_token.gs:
            yield InputToken(aligned_token.ocr, aligned_token.gs,
                             aligned_token.start, len(aligned_token.ocr), 0)
    else:
        parts = aligned_token.ocr.split(' ')
        new_start = aligned_token.start
        for i, part in enumerate(parts):
            if i == 0:
                yield InputToken(part, aligned_token.gs, aligned_token.start,
                                 len(part), 1)
            else:
                yield InputToken(part, '', new_start, len(part), 2)
            new_start += len(part) + 1

In [ ]:
t = AlignedToken('Long ow.', 'Longhow.', 'Long ow.', 'Longhow.', 24, 8)
print(t)

tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    print(inp_tok)
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ['Long', 'ow.']
assert labels == [1, 2]
assert ''.join(gs) == t.gs

In [ ]:
t = AlignedToken('INEVR', 'I NEVER', 'I@NEV@R', 'I NEVER', 0, 5)
print(t)

tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    print(inp_tok)
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ['INEVR']
assert labels == [1]
assert ''.join(gs) == t.gs

In [ ]:
t = AlignedToken('Major', 'Major', 'Major', 'Major', 19, 5)
print(t)

tokens = []
labels = []
gs = []

for inp_tok in get_input_tokens(t):
    print(inp_tok)
    tokens.append(inp_tok.ocr)
    labels.append(inp_tok.label)
    gs.append(inp_tok.gs)

assert tokens == ['Major']
assert labels == [0]
assert ''.join(gs) == t.gs

In [ ]:
#| export
def tokenize_aligned(ocr_aligned, gs_aligned):

    ocr_cursor = 0
    start = 0

    ocr_token_chars = []
    gs_token_chars = []
    ocr_token_chars_aligned = []
    gs_token_chars_aligned = []

    tokens = []

    for ocr_aligned_char, gs_aligned_char in zip(ocr_aligned, gs_aligned):
        #print(ocr_aligned_char, gs_aligned_char, ocr_cursor)
        # The # character in ocr is not an aligment character!
        if ocr_aligned_char != '@':
            ocr_cursor += 1

        if ocr_aligned_char == ' ' and gs_aligned_char == ' ':
            #print('TOKEN')
            #print('OCR:', repr(''.join(ocr_token_chars)))
            #print(' GS:', repr(''.join(gs_token_chars)))
            #print('start:', start_char)
            #ocr_cursor += 1

            # Ignore 'tokens' without representation in the ocr text
            # (these tokens do not consist of characters)
            ocr = (''.join(ocr_token_chars)).strip()
            if ocr != '':
                tokens.append(AlignedToken(ocr,
                                          ''.join(gs_token_chars),
                                          ''.join(ocr_token_chars_aligned),
                                          ''.join(gs_token_chars_aligned),
                                          start,
                                          len(''.join(ocr_token_chars))))
            start = ocr_cursor

            ocr_token_chars = []
            gs_token_chars = []
            ocr_token_chars_aligned = []
            gs_token_chars_aligned = []
        else:
            ocr_token_chars_aligned.append(ocr_aligned_char)
            gs_token_chars_aligned.append(gs_aligned_char)
            # The # character in ocr is not an aligment character!
            if ocr_aligned_char != '@':
                ocr_token_chars.append(ocr_aligned_char)
            if gs_aligned_char != '@' and gs_aligned_char != '#':
                gs_token_chars.append(gs_aligned_char)

    ocr = (''.join(ocr_token_chars)).strip()
    if ocr != '':
        tokens.append(AlignedToken(ocr,
                                   ''.join(gs_token_chars),
                                   ''.join(ocr_token_chars_aligned),
                                   ''.join(gs_token_chars_aligned),
                                   start,
                                   len(''.join(ocr_token_chars))))

    return tokens

In [ ]:
#| export
def window(iterable, size=2):
    i = iter(iterable)
    win = []
    for e in range(0, size):
        try:
            win.append(next(i))
        except StopIteration:
            break
    yield win
    for e in i:
        win = win[1:] + [e]
        yield win

In [ ]:
#| export
@dataclass
class Text:
    ocr_text: str
    tokens: list
    input_tokens: list
    score: float

In [ ]:
#| export
def clean(string):
    string = string.replace('@', '')
    string = string.replace('#', '')

    return string

In [ ]:
#| export
def process_text(in_file):
    with open(in_file) as f:
        lines = f.readlines()

    # The # character in ocr input is not an aligment character, but the @
    # character is!
    ocr_input = remove_label_and_nl(lines[0]).replace('@', '')
    ocr_aligned = remove_label_and_nl(lines[1])
    gs_aligned = remove_label_and_nl(lines[2])

    #print('ocr input:', ocr_input)
    #print('ocr aligned:', ocr_aligned)
    #print('gs aligned:',gs_aligned)

    tokens = tokenize_aligned(ocr_aligned, gs_aligned)

    # Check data
    for token in tokens:
        input_token = ocr_input[token.start:token.start+token.len_ocr]
        try:
            assert token.ocr == input_token.strip()
        except AssertionError:
            logger.warning(f'OCR != aligned OCR: Text: {str(in_file)}; ocr: {repr(token.ocr)}; ocr_input: {repr(input_token)}')
            raise

    ocr = clean(ocr_aligned)
    gs = clean(gs_aligned)

    try:
        ed = edlib.align(gs, ocr)['editDistance']
        score = normalized_ed(ed, ocr, gs)
    except UnicodeEncodeError:
        logger.warning(f'UnicodeEncodeError for text {in_file}; setting score to 1')
        score = 1

    input_tokens = []
    for token in tokens:
        for inp_tok in get_input_tokens(token):
            input_tokens.append(inp_tok)

    return Text(ocr_input, tokens, input_tokens, score)


In [ ]:
#| export
def generate_data(in_dir):

    data = {}

    file_languages = []
    file_names = []
    scores = []
    num_tokens = []
    num_input_tokens = []

    for language_dir in tqdm(in_dir.iterdir()):
        #print(language_dir.stem)
        language = language_dir.stem

        for text_file in language_dir.rglob('*.txt'):
            #print(text_file)
            #print(text_file.relative_to(in_dir))
            key = str(text_file.relative_to(in_dir))
            data[key] = process_text(text_file)

            file_languages.append(language)
            file_names.append(key)
            scores.append(data[key].score)
            num_tokens.append(len(data[key].tokens))
            num_input_tokens.append(len(data[key].input_tokens))
    md = pd.DataFrame({'language': file_languages,
                    'file_name': file_names,
                    'score': scores,
                    'num_tokens': num_tokens,
                    'num_input_tokens': num_input_tokens})
    return data, md


In [ ]:
#| export
def generate_sentences(df, data, size=15, step=10):
    sents = []
    labels = []
    keys = []
    start_tokens = []
    scores = []
    languages = []

    for idx, row in tqdm(df.iterrows()):
        key = row.file_name
        tokens = data[key].input_tokens

        # print(len(tokens))
        # print(key)
        for i, res in enumerate(window(tokens, size=size)):
            if i % step == 0:
                ocr = [t.ocr for t in res]
                lbls = [t.label for t in res]
                gs = []
                for t in res:
                    if t.gs != '':
                        gs.append(t.gs)
                ocr_str = ' '.join(ocr)
                gs_str = ' '.join(gs)
                ed = edlib.align(ocr_str, gs_str)['editDistance']
                score = normalized_ed(ed, ocr_str, gs_str)

                if len(ocr_str) > 0:

                    sents.append(ocr)
                    labels.append(lbls)
                    keys.append(key)
                    start_tokens.append(i)
                    scores.append(score)
                    languages.append(key[:2])
                else:
                    logger.info(f'Empty sample for text "{key}"')
                    logger.info(f'ocr_str: {ocr_str}')
                    logger.info(f'start token: {i}')
    data = pd.DataFrame({
        'key': keys,
        'start_token_id': start_tokens,
        'score': scores,
        'tokens': sents,
        'tags': labels,
        'language': languages
    })

    return data

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()